In [71]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
df = pd.read_json("scout_car.json",orient="recordings", lines=True)
pd.options.display.max_columns = 600
pd.options.display.max_rows = 6

In [72]:
#Functions :
#convert lists values to new columns with this no need to change column type to string or NAN to [] values normally requires before applying below.
def list_to_columns(df,col): 
    dftemp = pd.DataFrame([pd.Series(x) for x in df[col]]) # that is better and quicker. no need to run NAN_to_list.?
    column_names= []
    max_element = len(max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len))
    #max_element = len(max(df.loc[df[col].notna()][col], key=len))
    for i in range(max_element):
        column_names.append(col + str(i+1))
    dftemp.columns = column_names
    df2 = df.drop(col,axis=1)
    if max_element == 1:
        dftemp.rename(columns={col+'1': col},inplace = True)
    df_added = pd.concat([df2, dftemp], axis=1, sort=False)
    return df_added

# check a column if it contains list of values. returns 2 list that we need to define
def check_is_list(df):
    list_of_columns = []
    non_list_of_columns = []
    for i in df.columns:
        counter_list=0
        counter_non_list=0
        for j in df[i].values:
            if isinstance(j, list):
                counter_list+=1
            else:
                counter_non_list+=1
        if counter_list > 0:
            list_of_columns.append(i)
        else :
            non_list_of_columns.append(i)
    return list_of_columns,non_list_of_columns
#changes a phrase in a list to another value
def del_str_phrase(phrase,liste):
    while(phrase in liste) : 
        liste.remove(phrase)
        
#get the df colon without outliers specified
def get_df_wo_outlier(df,col,range=1.5):
    Q1 = df[col].quantile(q=0.25)
    Q3 = df[col].quantile(q=0.75)
    IQR = Q3-Q1
    return df[col][~((df[col] <  Q1-(range*IQR)) |(df[col] > Q3+(range*IQR))) ]

#to have a look at data, which columns contain what and how many.
def get_all_df(df):
    for i in df.columns:
        print(df[i].value_counts(dropna = False).rename_axis(i +' unique_values').reset_index(name=i +' counts'))

#quick look on a column        
def get_col_details(df,col):
    print("VALUE COUNTS")
    print(df[col].value_counts(dropna=False))
    print("-----------------------\n","UNIQUE VALUES")
    print(df[col].unique())
    print("-----------------------\n","NULL SUM")
    print(df[col].isnull().sum())
    print("-----------------------\n","NULL Percentage")
    print(df[col].isnull().sum()*100 /df[col].index.stop )
    print("-----------------------\n","COLUMN with NONE-NULL-NAN")
    print(df.query(f'{col} != {col}')[col])
    
#updates values NAN to empty list : []
def Nan_to_list(df,col):
    Nan_rows = df[col].isnull()
    df.loc[Nan_rows,col] = df.loc[Nan_rows,col].apply(lambda x : [])
    
def outlier_zscore(df, col, min_z=1, max_z = 5, step = 0.1, print_list = False):
    z_scores = zscore(df[col].dropna())
    threshold_list = []
    for threshold in np.arange(min_z, max_z, step):
        threshold_list.append((threshold, len(np.where(z_scores > threshold)[0])))
        df_outlier = pd.DataFrame(threshold_list, columns = ['threshold', 'outlier_count'])
        df_outlier['pct'] = (df_outlier.outlier_count - df_outlier.outlier_count.shift(-1))/df_outlier.outlier_count*100
    plt.plot(df_outlier.threshold, df_outlier.outlier_count)
    best_treshold = round(df_outlier.iloc[df_outlier.pct.argmax(), 0],2)
    outlier_limit = int(df[col].dropna().mean() + (df[col].dropna().std()) * df_outlier.iloc[df_outlier.pct.argmax(), 0])
    percentile_threshold = stats.percentileofscore(df[col].dropna(), outlier_limit)
    plt.vlines(best_treshold, 0, df_outlier.outlier_count.max(), 
               colors="r", ls = ":"
              )
    plt.annotate("Zscore : {}\nValue : {}\nPercentile : {}".format(best_treshold, outlier_limit, 
                                                                   (np.round(percentile_threshold, 3), 
                                                                    np.round(100-percentile_threshold, 3))), 
                 (best_treshold, df_outlier.outlier_count.max()/2))
    #plt.show()
    if print_list:
        print(df_outlier)
    return (plt, df_outlier, best_treshold, outlier_limit, percentile)

def outlier_inspect(df, col, min_z=1, max_z = 5, step = 0.5, max_hist = None, bins = 50):
    fig = plt.figure(figsize=(20, 6))
    fig.suptitle(col, fontsize=16)
    plt.subplot(1,3,1)
    if max_hist == None:
        sns.distplot(df[col], kde=False, bins = 50)
    else :
        sns.distplot(df[df[col]<=max_hist][col], kde=False, bins = 50)
    plt.subplot(1,3,2)
    sns.boxplot(df[col])
    plt.subplot(1,3,3)
    z_score_inspect = outlier_zscore(df, col, min_z=min_z, max_z = max_z, step = step)
    plt.show()

In [73]:
#!pip install pandas-profiling
#import pandas_profiling


In [74]:
#pandas_profiling.ProfileReport(df)

In [75]:
df.rename(columns=lambda x: x.replace("\n",""),inplace=True)

In [76]:
list(df.columns)

['url',
 'make_model',
 'short_description',
 'body_type',
 'price',
 'vat',
 'km',
 'registration',
 'prev_owner',
 'kW',
 'hp',
 'Type',
 'Previous Owners',
 'Next Inspection',
 'Inspection new',
 'Warranty',
 'Full Service',
 'Non-smoking Vehicle',
 'null',
 'Make',
 'Model',
 'Offer Number',
 'First Registration',
 'Body Color',
 'Paint Type',
 'Body Color Original',
 'Upholstery',
 'Body',
 'Nr. of Doors',
 'Nr. of Seats',
 'Model Code',
 'Gearing Type',
 'Displacement',
 'Cylinders',
 'Weight',
 'Drive chain',
 'Fuel',
 'Consumption',
 'CO2 Emission',
 'Emission Class',
 'Comfort & Convenience',
 'Entertainment & Media',
 'Extras',
 'Safety & Security',
 'description',
 'Emission Label',
 'Gears',
 'Country version',
 'Electricity consumption',
 'Last Service Date',
 'Other Fuel Types',
 'Availability',
 'Last Timing Belt Service Date',
 'Available from']

In [82]:
df['Next Inspection'].value_counts

<bound method IndexOpsMixin.value_counts of 0        [\n06/2021\n, \n99 g CO2/km (comb)\n]
1                                          NaN
2                                          NaN
                         ...                  
15916                                      NaN
15917                                      NaN
15918                                      NaN
Name: Next Inspection, Length: 15919, dtype: object>

In [83]:
df

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,Comfort & Convenience,Entertainment & Media,Extras,Safety & Security,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...","[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,85 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]",NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",[\nAM-95365\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nmythosschwarz metallic\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n4\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,598 cc\n]",NaN,NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Cruise control, Electrical ...","[MP3, On-board computer]","[Alloy wheels, Voice Control]","[ABS, Central door lock, Daytime running light...","[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15916,https://www.autoscout24.com//offers/renault-es...,Renault Espace,Blue dCi 200 EDC Initiale Paris Leder LED Navi...,Van,39875,VAT deductible,15 km,03/2019,1 previous owner,NaN,146 kW,"[, Pre-registered, , Diesel]","[\n1\n, \n139 g CO2/km (comb)\n]",NaN,"[\nYes\n, \nEuro 6d-TEMP\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nRenault\n,"

# 18sept

In [7]:
df["body_type"].value_counts(dropna = False)


Sedans           7903
Station wagon    3553
Compact          3153
                 ... 
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, Length: 10, dtype: int64

In [8]:
df[df["make_model"] == "Audi A3"]["body_type"].mode()[0]

'Sedans'

In [9]:
grp_inx = list(df[df['make_model'] == 'Audi A3']['body_type'].index)
grp_inx

[2615,
 2616,
 2617,
 2618,
 2619,
 2620,
 2621,
 2622,
 2623,
 2624,
 2625,
 2626,
 2627,
 2628,
 2629,
 2630,
 2631,
 2632,
 2633,
 2634,
 2635,
 2636,
 2637,
 2638,
 2639,
 2640,
 2641,
 2642,
 2643,
 2644,
 2645,
 2646,
 2647,
 2648,
 2649,
 2650,
 2651,
 2652,
 2653,
 2654,
 2655,
 2656,
 2657,
 2658,
 2659,
 2660,
 2661,
 2662,
 2663,
 2664,
 2665,
 2666,
 2667,
 2668,
 2669,
 2670,
 2671,
 2672,
 2673,
 2674,
 2675,
 2676,
 2677,
 2678,
 2679,
 2680,
 2681,
 2682,
 2683,
 2684,
 2685,
 2686,
 2687,
 2688,
 2689,
 2690,
 2691,
 2692,
 2693,
 2694,
 2695,
 2696,
 2697,
 2698,
 2699,
 2700,
 2701,
 2702,
 2703,
 2704,
 2705,
 2706,
 2707,
 2708,
 2709,
 2710,
 2711,
 2712,
 2713,
 2714,
 2715,
 2716,
 2717,
 2718,
 2719,
 2720,
 2721,
 2722,
 2723,
 2724,
 2725,
 2726,
 2727,
 2728,
 2729,
 2730,
 2731,
 2732,
 2733,
 2734,
 2735,
 2736,
 2737,
 2738,
 2739,
 2740,
 2741,
 2742,
 2743,
 2744,
 2745,
 2746,
 2747,
 2748,
 2749,
 2750,
 2751,
 2752,
 2753,
 2754,
 2755,
 2756,
 2757,

In [10]:
doldur = df[df["make_model"] == "Audi A3"]["body_type"].mode()[0]

In [11]:
df["body_type"].iloc[grp_inx] = df["body_type"].iloc[grp_inx].fillna(doldur)

In [12]:
df["body_type"].value_counts(dropna = False)



Sedans           7910
Station wagon    3553
Compact          3153
                 ... 
NaN                53
Coupe              25
Convertible         8
Name: body_type, Length: 10, dtype: int64

In [13]:
def fill_most(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        grp_inx = list(df[df[group_col]==group][col_name].index)
        df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[df[group_col]==group][col_name].mode()[0])

In [14]:
fill_most(df,"make_model","body_type")

In [15]:
#fill_most(df,"make_model","Extras")

In [16]:
#fill_most(df,"make_model","Comfort & Convenience")

In [17]:
df["body_type"].value_counts(dropna = False)


Sedans           7925
Station wagon    3563
Compact          3155
                 ... 
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, Length: 9, dtype: int64

# next inspection

In [18]:
"".join([i for i in df["Next Inspection"][0]])

'\n06/2021\n\n99 g CO2/km (comb)\n'

In [19]:
["".join(item).strip() if type(item)==list else item for item in df["Next Inspection" ] ]

['06/2021\n\n99 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '02/2020\n\n97 g CO2/km (comb)',
 nan,
 nan,
 nan,
 '09/2019\n\n112 g CO2/km (comb)',
 nan,
 '10/2019\n\n99 g CO2/km (comb)',
 '04/2020\n\n97 g CO2/km (comb)',
 nan,
 nan,
 '\n07/2020\n',
 nan,
 nan,
 nan,
 '12/2019\n\n112 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '05/2019\n\n95 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 '07/2019\n\n97 g CO2/km (comb)',
 '10/2019\n\n97 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 '02/2021\n\n94 g CO2/km (comb)',
 '06/2019\n\n99 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 '06/2019\n\n97 g CO2/km (comb)',
 nan,
 nan,
 nan,
 '05/2020\n\n102 g CO2/km (comb)',
 '11/2019\n\n97 g CO2/km (comb)',
 '06/2021\n\n102 g CO2/km (comb)',
 '12/2019\n\n99 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 '06/2021\n\n102 g CO2/km (comb)',
 nan,
 '\n11/2020\n',
 nan,
 nan,
 nan,
 '09/2021\n\n104 g CO2/km (comb)',
 nan,
 '03/2020\n\n117 g CO2/km (comb)',
 nan,
 na

In [20]:
#now use extract etc.

In [21]:
# car_acc_list = ['Comfort & Convenience',"Safety & Security","Extras","Entertainment & Media"]
# for col in car_acc_list:
#     listem = max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len)
#     for i in range(len(listem)):
#         df[f"{col} {str(i+1)} {listem[i]}"] = df[col].apply(lambda x : True if (type(x) == list and listem[i] in x) else np.nan) 


In [22]:
df.head(5)

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,Comfort & Convenience,Entertainment & Media,Extras,Safety & Security,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...","[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,85 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]",NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",[\nAM-95365\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nmythosschwarz metallic\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n4\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,598 cc\n]",NaN,NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Cruise control, Electrical ...","[MP3, On-board computer]","[Alloy wheels, Voice Control]","[ABS, Central door lock, Daytime running light...","[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]","[\n, \n, \nEuro 6\n]",[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2016, \n]","[\n, Brown, \n]",[\nMetallic\n],NaN,NaN,"[\n, Sedans, \n]",[\n3\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,195 kg\n]",NaN,"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (c

In [37]:
car_acc_list = ['Comfort & Convenience',"Safety & Security","Extras","Entertainment & Media"]
def list_to_feat_cols(df,col_list):
    dftemp = pd.DataFrame()
    for col in col_list:
        max_list = max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len)
        for i in range(len(max_list)):
            dftemp[f"{col} {str(i+1)} {max_list[i]}"] = df[col].apply(lambda x : True if (type(x) == list and max_list[i] in x) else False) 
        df = df.drop(col,axis=1)
    df_added = pd.concat([df, dftemp], axis=1, sort=False)
    return df_added

In [38]:
df3 = list_to_feat_cols(df,car_acc_list)

In [39]:
df3.head(2)

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from,Comfort & Convenience 1 Air conditioning,Comfort & Convenience 2 Armrest,Comfort & Convenience 3 Automatic climate control,Comfort & Convenience 4 Auxiliary heating,Comfort & Convenience 5 Cruise control,Comfort & Convenience 6 Electrically adjustable seats,Comfort & Convenience 7 Electrically heated windshield,Comfort & Convenience 8 Electrical side mirrors,Comfort & Convenience 9 Electric tailgate,Comfort & Convenience 10 Heads-up display,Comfort & Convenience 11 Heated steering wheel,Comfort & Convenience 12 Hill Holder,Comfort & Convenience 13 Keyless central door lock,Comfort & Convenience 14 Leather seats,Comfort & Convenience 15 Leather steering wheel,Comfort & Convenience 16 Light sensor,Comfort & Convenience 17 Lumbar support,Comfort & Convenience 18 Massage seats,Comfort & Convenience 19 Multi-function steering wheel,Comfort & Convenience 20 Navigation system,Comfort & Convenience 21 Panorama roof,Comfort & Convenience 22 Park Distance Control,Comfort & Convenience 23 Parking assist system camera,Comfort & Convenience 24 Parking assist system self-steering,Comfort & Convenience 25 Parking assist system sensors front,Comfort & Convenience 26 Parking assist system sensors rear,Comfort & Convenience 27 Power windows,Comfort & Convenience 28 Rain sensor,Comfort & Convenience 29 Seat heating,Comfort & Convenience 30 Seat ventilation,Comfort & Convenience 31 Split rear seats,Comfort & Convenience 32 Start-stop system,Comfort & Convenience 33 Tinted windows,Safety & Security 1 ABS,Safety & Security 2 Adaptive Cruise Control,Safety & Security 3 Adaptive headlights,Safety & Security 4 Alarm system,Safety & Security 5 Blind spot monitor,Safety & Security 6 Central door lock,Safety & Security 7 Central door lock with remote control,Safety & Security 8 Daytime running lights,Safety & Security 9 Driver drowsiness detection,Safety & Security 10 Driver-side airbag,Safety & Security 11 Electronic stability control,Safety & Security 12 Emergency brake assistant,Safety & Security 13 Emergency system,Safety & Security 14 Fog lights,Safety & Security 15 Head airbag,Safety & Security 16 Immobilizer,Safety & Security 17 Isofix,Safety & Security 18 Lane departure warning system,Safety & Security 19 LED Daytime Running Lights,Safety & Security 20 LED Headlights,Safety & Security 21 Night view assist,Safety & Security 22 Passenger-side airbag,Safety & Security 23 Power steering,Safety & Security 24 Rear airbag,Safety & Security 25 Side airbag,Safety & Security 26 Tire pressure monitoring system,Safety & Security 27 Traction control,Safety & Security 28 Traffic sign recognition,Safety & Security 29 Xenon headlights,Extras 1 Alloy wheels,Extras 2 Catalytic Converter,Extras 3 Shift paddles,Extras 4 Sport package,Extras 5 Sport seats,Extras 6 Sport suspension,Extras 7 Touch screen,Extras 8 Tuned car,Extras 9 Voice Control,Extras 10 Winter tyres,Entertainment & Media 1 Bluetooth,Entertainment & Media 2 CD player,Entertainment & Media 3 Digital radio,Entertainment & Media 4 Hands-free equipment,Entertainment & Media 5 MP3,Entertainment & Media 6 On-board computer,Entertainment & Media 7 Radio,Entertainment & Media 8 Sound system,Entertainment & Media 9 Television,Entertainment & Media 10 USB
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,

In [36]:
df3["Comfort & Convenience 16 Light sensor"].isnull().sum()

0

In [41]:
df4 = list_to_feat_cols(df,["Type"])

In [42]:
df4.head(2)

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,Comfort & Convenience,Entertainment & Media,Extras,Safety & Security,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from,Type 1,Type 2 Used,Type 3,Type 4 Diesel (Particulate Filter)
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...","[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,False


In [44]:
df["Type"]

0        [, Used, , Diesel (Particulate Filter)]
1                           [, Used, , Gasoline]
2        [, Used, , Diesel (Particulate Filter)]
                          ...                   
15916               [, Pre-registered, , Diesel]
15917               [, Pre-registered, , Diesel]
15918              [, Demonstration, , Super 95]
Name: Type, Length: 15919, dtype: object

In [66]:
col = "Type"
deneme = [["Used", "Diesel (Particulate Filter)"],["", "Used"," ", "Gasoline"]]
max_list = max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len)
ma = []
df[col].apply(lambda x : ma.append(x) if (type(x)==list and x not in ma) else x)


0                                           None
1                                           None
2        [, Used, , Diesel (Particulate Filter)]
                          ...                   
15916               [, Pre-registered, , Diesel]
15917               [, Pre-registered, , Diesel]
15918              [, Demonstration, , Super 95]
Name: Type, Length: 15919, dtype: object

In [67]:
ma

[['', 'Used', '', 'Diesel (Particulate Filter)'],
 ['', 'Used', '', 'Gasoline'],
 ['', 'Used', '', 'Super 95'],
 ['', 'Used', '', 'Regular/Benzine 91'],
 ['', "Employee's car", '', 'Diesel (Particulate Filter)'],
 ['', 'Used', '', 'Diesel'],
 ['',
  'Used',
  '',
  'Regular/Benzine 91 / Super Plus 98 / Regular/Benzine E10 91 / Super 95 / Super E10 95 / Super Plus E10 98'],
 ['', "Employee's car", '', 'Regular/Benzine 91'],
 ['', "Employee's car", '', 'Diesel'],
 ['', "Employee's car", '', 'Super E10 95'],
 ['', 'New', '', 'Super 95 (Particulate Filter)'],
 ['', 'Used', '', 'Super 95 / Regular/Benzine 91'],
 ['', "Employee's car", '', 'Super 95'],
 ['',
  'Used',
  '',
  'Super 95 / Super Plus 98 / Super E10 95 / Super Plus E10 98'],
 ['', 'Used', '', 'Super E10 95 / Super 95'],
 ['', 'Used', '', 'Super E10 95'],
 ['', 'Used', '', 'Super 95 / Regular/Benzine 91 / Super Plus 98'],
 ['', 'Demonstration', '', 'Super 95'],
 ['', 'Used', '', 'Super 95 / Super Plus 98 / Super E10 95'],
 ['', 

In [70]:
df ["Next Inspection"]

0        [\n06/2021\n, \n99 g CO2/km (comb)\n]
1                                          NaN
2                                          NaN
                         ...                  
15916                                      NaN
15917                                      NaN
15918                                      NaN
Name: Next Inspection, Length: 15919, dtype: object

In [100]:
z =[ ''.join(item).strip() if type(item) == list else item for item in [inner for outer in df["Consumption"] for inner in outer]]

TypeError: 'float' object is not iterable

In [99]:
z

['3.8 l/100 km (comb)', '4.3 l/100 km (city)', '3.5 l/100 km (country)']

In [97]:
df["consumption"].apply[lambda item :  ''.join(item).strip() if type(item) == list else item for item in [inner for outer in item for inner in outer]]

SyntaxError: invalid syntax (<ipython-input-97-d45217d6bd86>, line 1)

In [93]:
[inner for outer in df["Consumption"][0] for inner in outer]

['3.8 l/100 km (comb)', '4.3 l/100 km (city)', '3.5 l/100 km (country)']

In [85]:
z =[ ''.join(item).strip() if type(item) == list else item for item in df ["Consumption"]]

TypeError: sequence item 0: expected str instance, list found

In [88]:
"".join([i for i in for i in df["Consumption"][0]])

SyntaxError: invalid syntax (<ipython-input-88-6e8a40ed8a6f>, line 1)

In [91]:
z = [[j for j in range(5)] for i in range(5)]
z =[ ''.join(item).strip() if type(item) == list else item for item in df ["Consumption"]]

x = [i for i in df ["Consumption"][0]]

TypeError: sequence item 0: expected str instance, list found

In [ ]:
[ ''.join(item).strip() if type(item) == list else item for item in df ["Consumption"]]

In [102]:
z =[ ''.join(item).strip() if type(item) == list else item for item in [inner for outer in df["Consumption"]  for inner in outer if type(item) == list else item]]

SyntaxError: invalid syntax (<ipython-input-102-ee5fc360c650>, line 1)

In [101]:
z =[ item if pd.isna(item) else (''.join(item).strip() if type(item) == list else item for item in [inner for outer in df["Consumption"] for inner in outer])]

NameError: name 'item' is not defined